### Q1

In [1]:
! mlflow --version

mlflow, version 1.26.1


### Q2

In [3]:
! mkdir data

In [18]:
! wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
! wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
! wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet

--2022-05-29 19:25:11--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.78.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.78.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M  --.-KB/s    in 0.03s   

2022-05-29 19:25:11 (40.8 MB/s) - ‘green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-29 19:25:11--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.78.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.78.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘green_tripdata_2021-02.parquet’

green_tripdata_2021 100%[===================>]   1.09M  --.-KB/

In [19]:
! mv green_tripdata_2021-01.parquet data/green_tripdata_2021-01.parquet
! mv green_tripdata_2021-02.parquet data/green_tripdata_2021-02.parquet
! mv green_tripdata_2021-03.parquet data/green_tripdata_2021-03.parquet

In [20]:
! python homework/preprocess_data.py --raw_data_path ./data --dest_path ./output

In [25]:
! ls output

dv.pkl	test.pkl  train.pkl  valid.pkl


### Q3

#### Source code

In [26]:
import argparse
import os
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_valid)

    rmse = mean_squared_error(y_valid, y_pred, squared=False)


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    args = parser.parse_args()

    run(args.data_path)

usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/ubuntu/.local/share/jupyter/runtime/kernel-676cae7a-4b77-40d1-970b-974f48509a17.json


SystemExit: 2

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### With mlfow

In [27]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-homework")

2022/05/29 19:40:53 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-homework' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-homework', tags={}>

In [32]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-homework', tags={}>]

In [47]:
import argparse
import os
import pickle

import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(experiment_id: int, developer: str, data_path: str):
    
    mlflow.sklearn.autolog()
    
    with mlflow.start_run(experiment_id=experiment_id):
        
        mlflow.set_tag("developer", developer)

        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)

        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        
        
run("2", "Andrei", "./output")

- Parameters (17)

Name	Value

bootstrap	True

ccp_alpha	0.0

criterion	squared_error

max_depth	10

max_features	1.0

max_leaf_nodes	None

max_samples	None

min_impurity_decrease	0.0

min_samples_leaf	1

min_samples_split	2

min_weight_fraction_leaf	0.0

n_estimators	100

n_jobs	None

oob_score	False

random_state	0

verbose	0

warm_start	False

- Metrics

Name	Value

training_mae	3.958

training_mse	32.84

training_r2_score	0.754

training_rmse	5.731

training_score	0.754

#### Q4

default-artifact-root

#### Q5

In [6]:
%%writefile hpo_mlflow.py

import argparse
import os
import pickle

import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path, num_trials):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    def objective(params):
        
        with mlflow.start_run():
            mlflow.set_tag("model", "Random Forest Regressor")
            mlflow.log_params(params)
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_valid)
            rmse = mean_squared_error(y_valid, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    parser.add_argument(
        "--max_evals",
        default=50,
        help="the number of parameter evaluations for the optimizer to explore."
    )
    args = parser.parse_args()

    run(args.data_path, args.max_evals)

Overwriting hpo_mlflow.py


In [1]:
! python hpo_mlflow.py

100%|█████████| 50/50 [08:52<00:00, 10.65s/trial, best loss: 6.6284257482044735]


#### Q6

In [33]:
%%writefile register_model_hw.py

import argparse
import os
import pickle

import mlflow
from hyperopt import hp, space_eval
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-model"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()

SPACE = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 42
}


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        params = space_eval(SPACE, params)
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # evaluate model on the validation and test sets
        valid_rmse = mean_squared_error(y_valid, rf.predict(X_valid), squared=False)
        mlflow.log_metric("valid_rmse", valid_rmse)
        test_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
        mlflow.log_metric("test_rmse", test_rmse)


def run(data_path, log_top):

    client = MlflowClient()

    # retrieve the top_n model runs and log the models to MLflow
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=log_top,
        order_by=["metrics.rmse ASC"]
    )
    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        filter_string="",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.test_rmse ASC"]
    )[0]

    # register the best model
    mlflow.register_model(
        model_uri=f"runs:/{best_run.info.run_id}/model",
        name=EXPERIMENT_NAME
    )


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    parser.add_argument(
        "--top_n",
        default=5,
        type=int,
        help="the top 'top_n' models will be evaluated to decide which model to promote."
    )
    args = parser.parse_args()

    run(args.data_path, args.top_n)

Overwriting register_model_hw.py


In [34]:
! python register_model_hw.py

2022/05/31 10:01:01 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-model' does not exist. Creating a new experiment.
2022/05/31 10:01:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forest-best-model'.
2022/05/31 10:02:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best-model, version 1
Created version '1' of model 'random-forest-best-model'.


In [4]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-homework', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='random-forest-best-model', tags={}>]

In [5]:
experiment = client.get_experiment_by_name('random-forest-best-model')
best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        filter_string="",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.test_rmse ASC"]
    )[0]

In [12]:
round(best_run.data.metrics['test_rmse'], 2)

6.55